In [1]:
# NBA Free Agent Machine

In [1]:
# 1. Install and import packages
# ! pip install get_nba_data
# ! pip install nba_py
# ! pip install nba_api
# ! pip install sklearn
# ! pip install appmode
# ! jupyter nbextension     enable --py --sys-prefix appmode
# ! jupyter serverextension enable --py --sys-prefix appmode
import requests
import json
import pandas as pd
import numpy as np
import nba_api
import sklearn
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
# 2 Data Gathering and Manipulation
# 2.1 Connect to nba_api
from nba_api.stats.endpoints import leaguedashplayerstats

# 2.2 Gather NBA player data from 2018-19 season, 2017-18 season, and 2016-17 sesaon
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2018-19')
catstats2018_2019 = playerinfo.get_data_frames()
catstats2018_2019 = pd.concat(catstats2018_2019)
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2017-18')
catstats2017_2018 = playerinfo.get_data_frames()
catstats2017_2018 = pd.concat(catstats2017_2018)
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2016-17')
catstats2016_2017 = playerinfo.get_data_frames()
catstats2016_2017 = pd.concat(catstats2016_2017)

# 2.3 Remove unnecessary columns, add year to end of each column name
final2018_2019 = catstats2018_2019[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','GP','MIN','FGM','FGA',
'FG3M','FG3A','FTM','FTA','OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2018_2019 = final2018_2019.add_suffix('_2018')
final2017_2018 = catstats2017_2018[['PLAYER_ID','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2017_2018 = final2017_2018.add_suffix('_2017')
final2016_2017 = catstats2016_2017[['PLAYER_ID','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2016_2017 = final2016_2017.add_suffix('_2016')

# 2.4 Merge all dataframes to make one master dataframe with all 3 years of data
final = final2018_2019.merge(final2017_2018, left_on='PLAYER_ID_2018', right_on='PLAYER_ID_2017', how='left')
final = final.merge(final2016_2017, left_on='PLAYER_ID_2018', right_on='PLAYER_ID_2016', how='left')

# 2.5 Add new columns for FT made and missed, FG3 made and missed, FG2 made and missed
final['FTMiss_2018'] = final.apply(lambda row: row.FTA_2018 - row.FTM_2018, axis=1)
final['FTMiss_2017'] = final.apply(lambda row: row.FTA_2017 - row.FTM_2017, axis=1)
final['FTMiss_2016'] = final.apply(lambda row: row.FTA_2016 - row.FTM_2016, axis=1)
final['FG3Miss_2018'] = final.apply(lambda row: row.FG3A_2018 - row.FG3M_2018, axis=1)
final['FG3Miss_2017'] = final.apply(lambda row: row.FG3A_2017 - row.FG3M_2017, axis=1)
final['FG3Miss_2016'] = final.apply(lambda row: row.FG3A_2016 - row.FG3M_2016, axis=1)
final['FG2Made_2018'] = final.apply(lambda row: row.FGM_2018 - row.FG3M_2018, axis=1)
final['FG2A_2018'] = final.apply(lambda row: row.FGA_2018 - row.FG3A_2018, axis=1)
final['FG2Miss_2018'] = final.apply(lambda row: row.FG2A_2018 - row.FG2Made_2018, axis=1)
final['FG2Made_2017'] = final.apply(lambda row: row.FGM_2017 - row.FG3M_2017, axis=1)
final['FG2A_2017'] = final.apply(lambda row: row.FGA_2017 - row.FG3A_2017, axis=1)
final['FG2Miss_2017'] = final.apply(lambda row: row.FG2A_2017 - row.FG2Made_2017, axis=1)
final['FG2Made_2016'] = final.apply(lambda row: row.FGM_2016 - row.FG3M_2016, axis=1)
final['FG2A_2016'] = final.apply(lambda row: row.FGA_2016 - row.FG3A_2016, axis=1)
final['FG2Miss_2016'] = final.apply(lambda row: row.FG2A_2016 - row.FG2Made_2016, axis=1)

# 2.6 Drop unnecessary columns and rename
final.drop(['FG3A_2018', 'FG2A_2018', 'FTA_2018', 'FGM_2018', 'FGA_2018',
            'FG3A_2017', 'FG2A_2017', 'FTA_2017', 'FGM_2017', 'FGA_2017', 
            'FG3A_2016', 'FG2A_2016', 'FTA_2016', 'FGM_2016', 'FGA_2016', 
            'PLAYER_ID_2017', 'PLAYER_ID_2016'], axis=1, inplace=True)
final = final.rename(index=str,columns={'PLAYER_ID_2018': 'PLAYER_ID',
                                        'PLAYER_NAME_2018':'PLAYER',
                                        'TEAM_ABBREVIATION_2018':'TEAM',
                                        'FTM_2018':'FTMade_2018', 'FTM_2017':'FTMade_2017', 'FTM_2016':'FTMade_2016',
                                        'FG3M_2018':'FG3Made_2018', 'FG3M_2017':'FG3Made_2017', 'FG3M_2016':'FG3Made_2016'})

# 2.7 Use nba_api to gather player bio information
from nba_api.stats.endpoints import leaguedashplayerbiostats
playerinfo = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
PlayerBios = playerinfo.get_data_frames()
PlayerBios = pd.concat(PlayerBios)

# 2.8 Remove unnecessary columns, change the datatype of Draft to integer, merge with the final dataframe
PlayerBios = PlayerBios[['PLAYER_ID','AGE','PLAYER_HEIGHT_INCHES','PLAYER_WEIGHT','COUNTRY','DRAFT_NUMBER']]
PlayerBios['DRAFT_NUMBER'] = PlayerBios.DRAFT_NUMBER.replace('Undrafted','0')
PlayerBios['DRAFT_NUMBER'] = PlayerBios['DRAFT_NUMBER'].astype(str).astype(int)
PlayerBios = PlayerBios.rename(index=str,columns={'DRAFT_NUMBER': 'DRAFT'})
final = final.merge(PlayerBios, left_on='PLAYER_ID', right_on='PLAYER_ID', how='left')

# 2.9 Create CSV files for 2018 salaries, 2019 salaries, 2019 Free Agents with data from Spotrac.com
# 2.9.1 Read csv for 2018 NBA player salaries, merge with final dataframe
Sals2018 = pd.read_csv('2018Sals.csv', dtype={'CAP HIT':float})
Sals2018 = Sals2018.rename(index=str,columns={'CAP HIT': 'PREV_SAL'})
final = final.merge(Sals2018, left_on='PLAYER', right_on='PLAYER', how='inner')

# 2.9.2 Read csv for 2019 NBA player salaries, merge with final dataframe to create Signed Players dataframe
Sals2019 = pd.read_csv('2019Sals.csv', dtype={'CAP HIT':float})
Sals2019 = Sals2019.rename(index=str,columns={'CAP HIT': 'SAL'})
Signed = final.merge(Sals2019, left_on='PLAYER', right_on='PLAYER', how='inner')

# 2.9.3 Read csv for 2019 NBA Free Agents, merge with final dataframe to create Free Agents dataframe
FAs2019 = pd.read_csv('FAs2019.csv')
FAs2019 = FAs2019.rename(index=str,columns={'TEAM': 'PREV_TEAM'})
FAs = final.merge(FAs2019, left_on='PLAYER', right_on='PLAYER', how='inner')

# 2.10 Make the categorical variables dummy variables
# 2.10.1 If the Country is USA, change the variable to 1, if not then 0
Signed.loc[Signed['COUNTRY'] != 'USA', 'COUNTRY'] = '0'
Signed.loc[Signed['COUNTRY'] == 'USA', 'COUNTRY'] = '1'
FAs.loc[FAs['COUNTRY'] != 'USA', 'COUNTRY'] = '0'
FAs.loc[FAs['COUNTRY'] == 'USA', 'COUNTRY'] = '1'

# 2.10.2 Assign the number for each position (PG-1,SG-2,SF-3,PF-4,C-5)
Signed.loc[Signed['POS'] == 'PG', 'POS'] = '1'
Signed.loc[Signed['POS'] == 'SG', 'POS'] = '2'
Signed.loc[Signed['POS'] == 'SF', 'POS'] = '3'
Signed.loc[Signed['POS'] == 'PF', 'POS'] = '4'
Signed.loc[Signed['POS'] == 'C', 'POS'] = '5'
FAs.loc[FAs['POS'] == 'PG', 'POS'] = '1'
FAs.loc[FAs['POS'] == 'SG', 'POS'] = '2'
FAs.loc[FAs['POS'] == 'SF', 'POS'] = '3'
FAs.loc[FAs['POS'] == 'PF', 'POS'] = '4'
FAs.loc[FAs['POS'] == 'C', 'POS'] = '5'

# 2.11 Change datatypes to perform regression
Signed['COUNTRY'] = Signed.COUNTRY.astype(np.int64)
Signed['POS'] = Signed.POS.astype(np.int64)
Signed['DRAFT'] = Signed.DRAFT.astype(np.int64)
Signed['PLAYER_WEIGHT'] = Signed.PLAYER_WEIGHT.astype(np.int64)
FAs['COUNTRY'] = FAs.COUNTRY.astype(np.int64)
FAs['POS'] = FAs.POS.astype(np.int64)
FAs['DRAFT'] = FAs.DRAFT.astype(np.int64)
FAs['PLAYER_WEIGHT'] = FAs.PLAYER_WEIGHT.astype(np.int64)

# 2.12 Fill NA values with 0
Signed = Signed.fillna(0)
FAs = FAs.fillna(0)

# 2.13 Reorder Columns for Analysis
Signedcols = Signed.columns.tolist()
Signedcols = Signedcols[:3] + Signedcols[54:57] + Signedcols[3:54] + Signedcols[57:59]
Signed = Signed[Signedcols]
FAscols = FAs.columns.tolist()
FAscols = FAscols[:3] + FAscols[59:] + FAscols[54:57] + FAscols[3:54] + FAscols[57:58]
FAs = FAs[FAscols]

In [3]:
# 3 Random Forest Regression
# 3.1 Importing the dataset
X_train = Signed.iloc[:, 3:58].values
y_train = Signed.iloc[:, 58].values
X_test = FAs.iloc[:, 4:].values

# 3.2 Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 0)
regressor.fit(X_train, y_train)

# 3.3 Predicting a new result
y_pred = regressor.predict(X_test)
results = pd.DataFrame(FAs['PLAYER'])
results['y_pred'] = y_pred
results['y_pred'] = results.y_pred.astype(np.int64)
FAs['PRED SAL'] = results['y_pred']

In [ ]:
# 4 Output results
# 4.1 Define function to display dataframe of Signed Players
def viewSigned(SignedTeam):
    return Signed[Signed['TEAM'] == SignedTeam]

# 4.2 Create a dropdown menu for user to select team
teamSigned = widgets.Dropdown(
    options = list(sorted(Signed['TEAM'].unique())),
    description = 'Team: '
)

# 4.3 Define function to display dataframe of Free Agents
def viewFAs():
    return FAs

# 4.4 Using interact, display dataframes and dropdown menu
interact(viewSigned, SignedTeam = teamSigned)
interact(viewFAs)

# 4.5 Set salary cap limit and team salary given selected team
Sal_Cap = 101800000
Team_Sal = Signed[Signed['TEAM']==teamSigned.value]['SAL'].sum()

# 4.6 Loop indefinitely
while True:
# 4.7 Input a Free Agent
    FA_input = input("Select a Free Agent to sign: ")
# 4.8 If Free Agent input is "", break, if not, display a dataframe of just the selected free agent
    if (FA_input == ""):
        break
    else:
        display(FAs[FAs['PLAYER']==FA_input])
# 4.9 Input a salary offer
    Offered_Sal = int(input("How much money would you like to offer to %s? " % (FA_input)))
# 4.10 Iterate over team salary
    Team_Sal = Team_Sal + Offered_Sal
# 4.11 If the offer meets all requirements, print YES, if not, NO
    if (Team_Sal > Sal_Cap):
        print("NO")
    else:
        print("YES")

interactive(children=(Dropdown(description='Team: ', options=('ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL',…

interactive(children=(Output(),), _dom_classes=('widget-interact',))